<b><font size="5">Segmenting and Clustering of Toronto Neighborhood</font></b>

<b>1. Import Libraries </b>

In [2]:
#import libraries
import pandas as pd
import numpy as np

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#import folium to render map
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Imported!")

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Imported!


In [3]:
#Install beautifulsoup and html parser
!easy_install beautifulsoup4
!easy_install html5lib

Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4
Searching for html5lib
Best match: html5lib 0.999999999
Adding html5lib 0.999999999 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib


<b>2. Get the web page contents from the Wikipedia web site</b>

In [4]:
#Get page contents from the given URL
import requests
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#print(page.content)


<b>3. Use the HTML parser to extract table from the web page</b>

In [5]:
#Using BeautifulSoup to extract the page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

<p>
First, get contents in tags 'tr' from the table.
Next, extract the contents within the header tags 'th'
Last, extract the data within tags 'td' if Borough is not 'Not assigned'.
Change the value of Neighbourhood if its value is 'Not assigned'.
Save the data to a dataframe.
</p>

In [6]:
#Extract table from the page
html = list(soup.children)[2]
list(html.children)
body = list(html.children)[3]
body
canada_table = soup.find_all('table')[0]

list_tr = canada_table.find_all("tr")

#Extract header tags
header = []
tr = list_tr[0]
for th in tr:
    if(th != "\n"):
        header.append(th.get_text().rstrip())

#Read table contents and change Borough to Postcode if Borough is 'Not assigned'
records = []
for tr in list_tr:
    trs = tr.findAll("td")
    if trs:
        record = []
        record.append(trs[0].text)
        #Check if Borough is 'Not assigned'
        if(trs[1].text != "Not assigned"):
            record.append(trs[1].text)
            #Check if 'Neighbourhood if 'Not assigned'
            if(trs[2].text.rstrip() == "Not assigned"):
                record.append(trs[1].text)
            else:
                record.append(trs[2].text.rstrip())
            records.append(record)
    
#Create dataframe with table contents
canada_df = pd.DataFrame(data=records, columns = header)
print(canada_df.shape)
#print(canada_df)


(212, 3)


In [14]:
#Find rows with same Postcode and Borough and concat the Neighbourhood names 
#Create a new dataframe with these changes
list_canada = []
for (k1, k2), group in canada_df.groupby(['Postcode', 'Borough']):
    col = group['Neighbourhood']
    names = pd.Series(group['Neighbourhood'].values)
    grouped_names = names.str.cat(sep=',')
    new_row = [k1, k2, grouped_names]
    list_canada.append(new_row)

new_canada_df=pd.DataFrame(list_canada,columns=['Postcode','Borough', 'Neighbourhood'])
print(new_canada_df.shape)
print(new_canada_df.head(40))

(103, 3)
   Postcode       Borough                                      Neighbourhood
0       M1B   Scarborough                                      Rouge,Malvern
1       M1C   Scarborough               Highland Creek,Rouge Hill,Port Union
2       M1E   Scarborough                    Guildwood,Morningside,West Hill
3       M1G   Scarborough                                             Woburn
4       M1H   Scarborough                                          Cedarbrae
5       M1J   Scarborough                                Scarborough Village
6       M1K   Scarborough          East Birchmount Park,Ionview,Kennedy Park
7       M1L   Scarborough                      Clairlea,Golden Mile,Oakridge
8       M1M   Scarborough      Cliffcrest,Cliffside,Scarborough Village West
9       M1N   Scarborough                         Birch Cliff,Cliffside West
10      M1P   Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...
11      M1R   Scarborough                                   Maryval